In [1]:
import sys
import os
import numpy as np

from codingTools import *

In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
#originalFile="../images/03508649.1_512_512_2_0_12_0_0_0.raw" #int16
originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
ImageRaw = load_image_raw(originalFile, 1, 2560, 2048, "uint8") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [3]:
########STATISTICS##########
statistics = statistics(original_image)
entropy = statistics.compute_entropy()
print("order zero entropy of the image: ",entropy)
print('hola')

order zero entropy of the image:  7.426142695578908
hola
